In [1]:
import sys
import os
import random

# Add the absolute path to the src directory 
os.chdir('../src')
src_path = os.path.abspath('../src/')
sys.path.insert(0, src_path)

from setup import *
seed = 123
import random
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

All working on Pepties Func dataset

In [ ]:
# load the data
mp.set_start_method('spawn', force=True)
# Load entire Peptides-func dataset
train_loader, test_loader, num_classes = load_peptides_func(parallel=True, subset_ratio=1.0)
print('Data done loading')
num_node_features = 9 

Loaded and split data
Data done loading


# GPS

In [3]:
# Define model, optimizer, and scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPSLongRange(
    num_node_features=num_node_features,
    hidden_channels=32,
    num_classes=num_classes,
    num_layers=5
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
print('Starting training')

# Training model, print final results
for epoch in range(200):
    train_loss = longrange_train(model, train_loader, optimizer, device)
    train_acc = longrange_test(model, train_loader, device)
    test_acc = longrange_test(model, test_loader, device)
    if epoch % 10 == 0:
        print(f"EPOCH {epoch} : Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")
        
    scheduler.step(train_loss)
print(f"FINAL: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")

Starting training
EPOCH 0 : Train Loss = 1.5592, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 10 : Train Loss = 1.3358, Train Acc = 0.5801 , Test Acc = 0.5894
EPOCH 20 : Train Loss = 1.2685, Train Acc = 0.6043 , Test Acc = 0.6221
EPOCH 30 : Train Loss = 1.2119, Train Acc = 0.6234 , Test Acc = 0.6414
EPOCH 40 : Train Loss = 1.2068, Train Acc = 0.6182 , Test Acc = 0.6221
EPOCH 50 : Train Loss = 1.1773, Train Acc = 0.6108 , Test Acc = 0.6285
EPOCH 60 : Train Loss = 1.1706, Train Acc = 0.6181 , Test Acc = 0.6244
EPOCH 70 : Train Loss = 1.1469, Train Acc = 0.6370 , Test Acc = 0.6497
EPOCH 80 : Train Loss = 1.1208, Train Acc = 0.6428 , Test Acc = 0.6418
EPOCH 90 : Train Loss = 1.1159, Train Acc = 0.6482 , Test Acc = 0.6464
EPOCH 100 : Train Loss = 1.1106, Train Acc = 0.6460 , Test Acc = 0.6506
EPOCH 110 : Train Loss = 1.1034, Train Acc = 0.6556 , Test Acc = 0.6524
EPOCH 120 : Train Loss = 1.0986, Train Acc = 0.6535 , Test Acc = 0.6561
EPOCH 130 : Train Loss = 1.0903, Train Acc = 0.6574 , Tes

# GCN

In [ ]:
# Define model, optimizer, and scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCNGraph(
    num_node_features=num_node_features,
    hidden_channels=32,
    num_classes=num_classes,
    layers=3
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
print('Starting training')

# Training model, print final results
for epoch in range(80):
    train_loss = longrange_train(model, train_loader, optimizer, device)
    train_acc = longrange_test(model, train_loader, device)
    test_acc = longrange_test(model, test_loader, device)
    if epoch % 10 == 0:
        print(f"EPOCH {epoch} : Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")
        
    scheduler.step(train_loss)
print(f"FINAL: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")

Starting training
EPOCH 0 : Train Loss = 1.6090, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 10 : Train Loss = 1.3831, Train Acc = 0.5208 , Test Acc = 0.5421
EPOCH 20 : Train Loss = 1.3819, Train Acc = 0.5293 , Test Acc = 0.5549
EPOCH 30 : Train Loss = 1.3692, Train Acc = 0.5330 , Test Acc = 0.5577
EPOCH 40 : Train Loss = 1.3420, Train Acc = 0.5673 , Test Acc = 0.5862
EPOCH 50 : Train Loss = 1.3311, Train Acc = 0.5778 , Test Acc = 0.5945
EPOCH 60 : Train Loss = 1.3095, Train Acc = 0.5921 , Test Acc = 0.6074
EPOCH 70 : Train Loss = 1.2929, Train Acc = 0.5888 , Test Acc = 0.6055
FINAL: Train Loss = 1.2838, Train Acc = 0.6023 , Test Acc = 0.6170


# GIN

In [8]:
# Define model, optimizer, and scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GINGraph(
    in_channels=num_node_features,
    hidden_channels=32,
   out_channels=num_classes,
   num_layers=3 
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
print('Starting training')

# Training model, print final results
for epoch in range(50):
    train_loss = longrange_train(model, train_loader, optimizer, device)
    train_acc = longrange_test(model, train_loader, device)
    test_acc = longrange_test(model, test_loader, device)
    if epoch % 10 == 0:
        print(f"EPOCH {epoch} : Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")
    scheduler.step(train_loss)
print(f"FINAL: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")

Starting training
EPOCH 0 : Train Loss = 1.2809, Train Acc = 0.5736 , Test Acc = 0.5729
EPOCH 10 : Train Loss = 1.0872, Train Acc = 0.5732 , Test Acc = 0.5816
EPOCH 20 : Train Loss = 1.0691, Train Acc = 0.5758 , Test Acc = 0.5876
EPOCH 30 : Train Loss = 1.0680, Train Acc = 0.6099 , Test Acc = 0.6207
EPOCH 40 : Train Loss = 1.0531, Train Acc = 0.6015 , Test Acc = 0.6106
FINAL: Train Loss = 1.0397, Train Acc = 0.6662 , Test Acc = 0.6777


# GAT

In [ ]:
# Define model, optimizer, and scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GATGraph(
    in_channels=num_node_features,
    hidden_channels=32,
    out_channels=num_classes,
    heads = 2,
    layers= 3
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
print('Starting training')

# Training model, print final results
for epoch in range(80):
    train_loss = longrange_train(model, train_loader, optimizer, device)
    train_acc = longrange_test(model, train_loader, device)
    test_acc = longrange_test(model, test_loader, device)
    if epoch % 10 == 0:
        print(f"EPOCH {epoch} : Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")
    scheduler.step(train_loss)
    
print(f"FINAL: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")

Starting training
EPOCH 0 : Train Loss = 2.1035, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 10 : Train Loss = 1.6043, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 20 : Train Loss = 1.5895, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 30 : Train Loss = 1.5819, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 40 : Train Loss = 1.5752, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 50 : Train Loss = 1.5648, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 60 : Train Loss = 1.5615, Train Acc = 0.5105 , Test Acc = 0.5320
EPOCH 70 : Train Loss = 1.5592, Train Acc = 0.5105 , Test Acc = 0.5320
FINAL: Train Loss = 1.5590, Train Acc = 0.5105 , Test Acc = 0.5320
